In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import os
import imblearn

In [2]:
main_dir = 'D:/Competitions/datastorm-2021/'


In [3]:
data_dir = main_dir+'data/feature/'
train_file = data_dir+'cbasi_train.csv'
val_file = data_dir+'cbasi_validation.csv'
test_file = data_dir+'cbasi_test.csv'
NN_dir = main_dir+'src/scripts/NeuralNet/'

In [4]:
train_ds = pd.read_csv(train_file)
val_ds = pd.read_csv(val_file)


In [5]:
def start_category_from_zero(df,columns):
    for each in columns:
        df[each] = df[each] - 1

    return df


In [6]:
start_category_from_zero(train_ds,['Reservation_Status'])
start_category_from_zero(val_ds,['Reservation_Status'])

,Age,Ethnicity,Educational_Level,Income,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Required_Car_Parking,Use_Promotion,Room_Rate,month_Expected_checkin,dayofweek_Expected_checkin,stay,total_participants,Reservation_Status
0,0.785640,-0.017315,-0.001419,0.007453,0.145374,0.010791,0.003794,0.022095,-0.014993,0.004285,0.384342,8,2,2,4,2
1,1.047029,0.010376,-0.003171,0.012407,0.622649,-0.021917,0.003794,0.022095,0.005952,-0.013100,0.270386,8,2,4,4,1
2,0.916334,-0.009340,-0.003171,0.007453,0.622649,0.010791,0.003794,0.022095,-0.014993,0.004285,1.182040,9,3,1,4,1
3,-1.370818,0.010376,-0.003171,0.012407,0.622649,-0.021917,0.003794,-0.069188,-0.014993,0.004285,0.315968,8,2,2,3,0
4,0.197515,-0.009340,-0.003171,0.012407,0.145374,-0.021917,0.003794,0.022095,0.005952,0.004285,0.976918,8,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744,-1.436166,-0.017315,-0.003171,-0.012088,-0.444474,-0.021917,0.003794,-0.069188,-0.014993,0.004285,0.543882,11,4,3,7,0
2745,-0.586652,-0.017315,-0.003171,0.012407,0.622649,0.010791,0.003794,0.025248,0.005952,0.004285,-1.575714,12,4,1,5,1
2746,-1.697554,-0.009340,-0.001419,0.012407,0.622649,0.010791,0.003794,-0.069188,-0.014993,0.004285,-0.891973,11,4,2,5,2
2747,-0.129221,-0.009340,-0.003171,0.007453,0.145374,-0.021917,-0.030540,0.022095,0.005952,0.004285,1.159249,11,4,1,6,0


In [7]:
print(train_ds['Reservation_Status'].value_counts())
train_ds.dtypes

0    21240
1    21240
2    21240
Name: Reservation_Status, dtype: int64


Age                           float64
Ethnicity                     float64
Educational_Level             float64
Income                        float64
Meal_Type                     float64
Visted_Previously             float64
Previous_Cancellations        float64
Deposit_type                  float64
Required_Car_Parking          float64
Use_Promotion                 float64
Room_Rate                     float64
month_Expected_checkin          int64
dayofweek_Expected_checkin      int64
stay                            int64
total_participants              int64
Reservation_Status              int64
dtype: object

In [8]:
x_columns = [x for x in train_ds.columns if x != 'Reservation_Status']
y_columns = ['Reservation_Status']
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)
batch_size = 64
input_shape = (len(x_columns))
epochs = 200
print(input_shape)

15


In [9]:
x_columns

['Age',
 'Ethnicity',
 'Educational_Level',
 'Income',
 'Meal_Type',
 'Visted_Previously',
 'Previous_Cancellations',
 'Deposit_type',
 'Required_Car_Parking',
 'Use_Promotion',
 'Room_Rate',
 'month_Expected_checkin',
 'dayofweek_Expected_checkin',
 'stay',
 'total_participants']

In [10]:
x_train = np.asarray(train_ds[x_columns])
y_train = np.asarray(train_ds[y_columns])
x_val = np.asarray(val_ds[x_columns])
y_val = np.asarray(val_ds[y_columns])

In [11]:
x_train.shape

(63720, 15)

In [12]:
y_train.shape

(63720, 1)

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
x_sm, y_train = sm.fit_resample(x_train, y_train)
x_train = pd.DataFrame(x_sm,columns=x_columns)

In [14]:
x_train.shape

(63720, 15)

In [15]:
y_train.shape

(63720,)

In [16]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
model.add(tf.keras.layers.Dense(100,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(150,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(150,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.6))
model.add(tf.keras.layers.Dense(150,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(3,activation='softmax'))


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1600      
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 150)               15150     
_________________________________________________________________
batch_normalization_1 (Batch (None, 150)               600       
_________________________________________________________________
dropout (Dropout)            (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 150)               22650     
_________________________________________________________________
batch_normalization_2 (Batch (None, 150)               6

In [18]:
def func(x):
    return x.argmax()

In [19]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

validation_data = (x_val,y_val)

class Metrics(Callback):
    def on_train_begin(self,logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(
            validation_data[0])))
        val_predict = np.apply_along_axis(func, 1, val_predict)
        val_targ = validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict,average='macro')
        _val_recall = recall_score(val_targ, val_predict,average='macro')
        _val_precision = precision_score(val_targ, val_predict,average='macro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(f" — val_f1: {_val_f1} — val_precision: {_val_precision} — val_recall _val_recall")
        return


metrics = Metrics()

In [20]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer= optimizer,
    metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train ,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data = validation_data,
                    callbacks=[metrics])

Epoch 1/200
990/996 [============================>.] - ETA: 0s - loss: 1.2057 - accuracy: 0.3666

In [ ]:
prediction_val = model.predict(x_val)
prediction_train  = model.predict(x_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
prediction_train = np.apply_along_axis(func, 1, prediction_train)
prediction_val = np.apply_along_axis(func, 1, prediction_val)

In [ ]:
y_train

In [ ]:
prediction_train = np.apply_along_axis(func, 1, prediction_train)
prediction_val = np.apply_along_axis(func, 1, prediction_val)

In [ ]:
y_train

In [ ]:
stats =  {
    'train' : {
        'accuracy' : accuracy_score(y_train, prediction_train),
        'precision' : precision_score(y_train,prediction_train, average='macro'),
        'recall' : recall_score(y_train, prediction_train, average='macro'),
        'f1-score' : f1_score(y_train, prediction_train, average='macro')},
    'test' : {
        'accuracy' : accuracy_score(y_val, prediction_val),
        'precision' : precision_score(y_val, prediction_val, average='macro'),
        'recall' : recall_score(y_val, prediction_val, average='macro'),
        'f1-score' : f1_score(y_val, prediction_val, average='macro')
    }
}

In [ ]:
print(stats)
with open(NN_dir+'score_BEFOREencode_cb.json','w') as pf:
    json.dump(stats, pf)

In [ ]:
import matplotlib.pyplot as plt
def acc_n_loss(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    with open("metrics.json", 'w') as outfile:
        json.dump({"Training-accuracy": acc[-1], "Validation-accuracy": val_acc[-1], "Training-loss": loss[-1],
                   "Validation-loss": val_loss[-1]}, outfile)

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
acc_n_loss(history)

In [ ]:
test_ds = pd.read_csv(test_file)

In [ ]:
test_ds

In [ ]:
assert list(test_ds.iloc[:,1:].columns) == x_columns

In [ ]:
x_test = np.asarray(test_ds.iloc[:,1:])

In [ ]:
x_test

In [ ]:
prediction_test = model.predict(x_test)

In [ ]:
prediction_test = np.apply_along_axis(func, 1, prediction_test)

In [ ]:
prediction_test

In [ ]:
sub_df = pd.DataFrame()
sub_df['Reservation-id'] = test_ds.iloc[:, 0]
sub_df['Reservation_status'] = prediction_test

In [ ]:
if 'predicts' not in os.listdir(NN_dir):
    os.mkdir(NN_dir+'predicts/')

In [ ]:
sub_df.to_csv(NN_dir+'predicts/'+str(stats['test']['f1-score'])+'_.csv')